In [2]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
import re

modele = load_model("modele_de_base_petit.h5")
csv_file = "dataQuestionnaire.csv"
data = pd.read_csv(csv_file)

In [3]:
print(data.shape)

(741, 90)


In [4]:
# Liste des colonnes avec commentaires
texte_colonnes = [
    "Commentaires éventuels (sur le formulaire ou sur votre situation personnelle)",
    "Commentaires éventuels (sur le formulaire ou sur votre appréciation personnelle de l'enseignement)",
    "Commentaires éventuels (sur le formulaire ou sur votre appréciation personnelle de la recherche)",
    "Pouvez-vous expliquer vos réponses merci ..."
]

# Calcul des nombres de valeurs non nulles pour chaque colonne avec commentaires
for colonne in texte_colonnes:
    nb_non_null = data[colonne].count()
    print(f"Nombre de valeurs non nulles dans la colonne '{colonne}': {nb_non_null}")


Nombre de valeurs non nulles dans la colonne 'Commentaires éventuels (sur le formulaire ou sur votre situation personnelle)': 104
Nombre de valeurs non nulles dans la colonne 'Commentaires éventuels (sur le formulaire ou sur votre appréciation personnelle de l'enseignement)': 183
Nombre de valeurs non nulles dans la colonne 'Commentaires éventuels (sur le formulaire ou sur votre appréciation personnelle de la recherche)': 185
Nombre de valeurs non nulles dans la colonne 'Pouvez-vous expliquer vos réponses merci ...': 366


In [5]:
# Fonction pour nettoyer le texte
def nettoyer_texte(texte):
    if isinstance(texte, str):
        texte = texte.lower()
        texte = re.sub("&lt;/?.*?&gt;", " &lt;&gt; ", texte)
        texte = re.sub(r'[@/#]', '', texte)
        texte = re.sub("(\\d|\\W)+", " ", texte)
        texte = re.sub(r'_+', '', texte)
        texte = re.sub(r'[^a-zA-ZÀ-ÿ\s]', '', texte)
        texte = re.sub(r'http\S+', '', texte)
        texte = re.sub(r'\s+', ' ', texte).strip()
    else:
        texte = str(texte)
    return texte

MAX_SEQUENCE_LENGTH = 100

# Fonction pour prédire la note
def predire_note(commentaire):
    if isinstance(commentaire, str):
        try:
            commentaire_seq = tokenizer.texts_to_sequences([commentaire])
            commentaire_pad = pad_sequences(commentaire_seq, maxlen=MAX_SEQUENCE_LENGTH)
            note = modele.predict(commentaire_pad)[0][0]
            return note
        except:
            return np.nan
    else:
        return np.nan

In [6]:
colonnes = ["SID", "Commentaires éventuels (sur le formulaire ou sur votre situation personnelle)"]
personnelle = data[colonnes].copy()
personnelle = personnelle.dropna(subset=[colonnes[1]])

tokenizer = Tokenizer()
tokenizer.fit_on_texts(personnelle[colonnes[1]])

personnelle[colonnes[1]] = personnelle[colonnes[1]].apply(nettoyer_texte)
personnelle["Note personnelle"] = personnelle[colonnes[1]].apply(predire_note)

print(personnelle.head())
print(personnelle["Note personnelle"].value_counts())

1/1 [==============================] - 0s 75ms/step
         SID  \
1   15896547   
10  15897993   
14  15899015   
15  15899810   
20  15901382   

   Commentaires éventuels (sur le formulaire ou sur votre situation personnelle)  \
1   il n y a plus d ufr à l université de bordeaux...                              
10                   temps moyen hors période bouchon                              
14  temps moyen de déplacement selon les sites peu...                              
15  je suis responsable de m mon travail a une for...                              
20  pas de distance en temps pour transiter du lie...                              

    Note personnelle  
1           0.776468  
10          0.453206  
14          0.390447  
15          0.709486  
20          0.200862  
0.579345    4
0.513245    3
0.482969    2
0.633834    1
0.212268    1
           ..
0.767121    1
0.914573    1
0.279605    1
0.610614    1
0.107040    1
Name: Note personnelle, Length: 98, dtype: int64


In [7]:
print(data.shape)

(741, 90)


In [8]:
colonnes2 = ["SID", "Commentaires éventuels (sur le formulaire ou sur votre appréciation personnelle de l'enseignement)"]
enseignement = data[colonnes2].copy()
enseignement = enseignement.dropna(subset=[colonnes2[1]])

enseignement[colonnes2[1]] = enseignement[colonnes2[1]].apply(nettoyer_texte)
enseignement["Note enseignement"] = enseignement[colonnes2[1]].apply(predire_note)

print(enseignement["Note enseignement"].value_counts())

1/1 [==============================] - 0s 45ms/step
0.048162    1
0.279725    1
0.391693    1
0.579345    1
0.165069    1
           ..
0.326526    1
0.596382    1
0.517298    1
0.131380    1
0.254598    1
Name: Note enseignement, Length: 183, dtype: int64


In [9]:
enseignement.head(20)

,SID,Commentaires éventuels (sur le formulaire ou sur votre appréciation personnelle de l'enseignement),Note enseignement
4,15896653,l enseignement est de plus en plus difficile j...,0.048162
10,15897993,a ma connaissance mon enseignement n est pas é...,0.682070
13,15898762,attention pour être reconnu l investissement p...,0.157585
14,15899015,dernière question s il y a et non pas s il n y...,0.695915
15,15899810,quand un jeune me demande si devenir prof de f...,0.478117
18,15900109,ce n est pas qu un probleme de nb d heures mai...,0.202384
25,15902841,je bénéficie d une décharge financer sur proje...,0.098017
34,15903256,appréciation personnelle aucune option ne me c...,0.438109
35,15903369,la très forte augmentation du nombre d étudian...,0.098765
43,15904144,j aime toujours enseigner mais un simple servi...,0.755182


In [10]:
colonnes3 = ["SID", "Commentaires éventuels (sur le formulaire ou sur votre appréciation personnelle de la recherche)"]
recherche = data[colonnes3].copy()
recherche = recherche.dropna(subset=[colonnes3[1]])

recherche[colonnes3[1]] = recherche[colonnes3[1]].apply(nettoyer_texte)
recherche["Note recherche"] = recherche[colonnes3[1]].apply(predire_note)

print(recherche["Note recherche"].value_counts())

1/1 [==============================] - 0s 39ms/step
0.579345    3
0.518360    1
0.829845    1
0.431217    1
0.510244    1
           ..
0.574282    1
0.840253    1
0.308604    1
0.730304    1
0.492824    1
Name: Note recherche, Length: 183, dtype: int64


In [11]:
colonnes4 = ["SID", "Pouvez-vous expliquer vos réponses merci ..."]
merci = data[colonnes4].copy()
merci = merci.dropna(subset=[colonnes4[1]])

merci[colonnes4[1]] = merci[colonnes4[1]].apply(nettoyer_texte)
merci["Note merci"] = merci[colonnes4[1]].apply(predire_note)

print(merci["Note merci"].value_counts())

1/1 [==============================] - 0s 45ms/step
0.279350    1
0.593635    1
0.385865    1
0.078346    1
0.252416    1
           ..
0.303829    1
0.906902    1
0.365155    1
0.632910    1
0.154763    1
Name: Note merci, Length: 366, dtype: int64


In [12]:
nouvelle_df = data[["SID"]].copy()
nouvelle_df["Totalite des commentaires"] = data[texte_colonnes].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1)
nouvelle_df = nouvelle_df.dropna(subset=["Totalite des commentaires"])
nouvelle_df = nouvelle_df[nouvelle_df["Totalite des commentaires"].apply(lambda x: x.strip() != "")]

print(nouvelle_df.head())
nouvelle_df["Totalite des commentaires"] = nouvelle_df["Totalite des commentaires"].apply(nettoyer_texte)
nouvelle_df["Note totalite"] = nouvelle_df["Totalite des commentaires"].apply(predire_note)

print(nouvelle_df["Note totalite"].value_counts())

        SID                          Totalite des commentaires
1  15896547  Il n'y a plus d'UFR à l'Université de Bordeaux...
2  15896551  Les conditions de travail dépendent beaucoup d...
4  15896653  L'enseignement est de plus en plus difficile. ...
5  15896801  Je suis dans une équipe Inria. Cela offre des ...
7  15897130  Aujourd’hui, l’informatique offre une multitud...
1/1 [==============================] - 0s 45ms/step
0.579345    2
0.776468    1
0.369187    1
0.235780    1
0.775790    1
           ..
0.592660    1
0.143490    1
0.551267    1
0.026781    1
0.154763    1
Name: Note totalite, Length: 408, dtype: int64


In [13]:
merged_df = pd.merge(personnelle, enseignement, on='SID', how='outer')
merged_df = pd.merge(merged_df, recherche, on='SID', how='outer')
merged_df = pd.merge(merged_df, merci, on='SID', how='outer')
merged_df = pd.merge(merged_df, nouvelle_df, on='SID', how='outer')

In [14]:
merged_df.head(15)

,SID,Commentaires éventuels (sur le formulaire ou sur votre situation personnelle),Note personnelle,Commentaires éventuels (sur le formulaire ou sur votre appréciation personnelle de l'enseignement),Note enseignement,Commentaires éventuels (sur le formulaire ou sur votre appréciation personnelle de la recherche),Note recherche,Pouvez-vous expliquer vos réponses merci ...,Note merci,Totalite des commentaires,Note totalite
0,15896547,il n y a plus d ufr à l université de bordeaux...,0.776468,NaN,NaN,NaN,NaN,NaN,NaN,il n y a plus d ufr à l université de bordeaux...,0.776468
1,15897993,temps moyen hors période bouchon,0.453206,a ma connaissance mon enseignement n est pas é...,0.682070,auparavant au cea r d migration université pou...,0.859789,profession en perte de considération dans la s...,0.560461,temps moyen hors période bouchon a ma connaiss...,0.581256
2,15899015,temps moyen de déplacement selon les sites peu...,0.390447,dernière question s il y a et non pas s il n y...,0.695915,NaN,NaN,on peut faire notre recherche comme on le souh...,0.381558,temps moyen de déplacement selon les sites peu...,0.382018
3,15899810,je suis responsable de m mon travail a une for...,0.709486,quand un jeune me demande si devenir prof de f...,0.478117,j ai arrêté pour les raisons indiquées dans mo...,0.608962,non sans hésitation aux trois questions voir m...,0.438614,je suis responsable de m mon travail a une for...,0.508164
4,15901382,pas de distance en temps pour transiter du lie...,0.200862,NaN,NaN,j aime toujours faire de la recherche mais j a...,0.505255,métier intéressant car très varié et avec des ...,0.548022,pas de distance en temps pour transiter du lie...,0.483975
5,15902826,ufr d informatique propre composée à d informa...,0.695436,NaN,NaN,NaN,NaN,admin car la plupart du temps c est de l admin...,0.382916,ufr d informatique propre composée à d informa...,0.389705
6,15903001,je ne fais pratiquement plus ma recherche dans...,0.647195,NaN,NaN,NaN,NaN,la charge administrative que j ai choisie n ét...,0.502141,je ne fais pratiquement plus ma recherche dans...,0.502388
7,15903160,enseignement sur plusieurs sites de la ville l...,0.713982,NaN,NaN,NaN,NaN,la liberté intellectuelle apportée par le vole...,0.487186,enseignement sur plusieurs sites de la ville l...,0.527541
8,15903369,une partie de l enseignement sur le même lieu ...,0.664801,la très forte augmentation du nombre d étudian...,0.098765,le manque de temps à consacrer à la recherche ...,0.586488,depuis ans je n ai pas eu le temps de déposer ...,0.153603,une partie de l enseignement sur le même lieu ...,0.153715
9,15904052,sans hdr par choix de ne pas rédiger ce docume...,0.173181,NaN,NaN,NaN,NaN,le métier se dégrade nous faisons de plus en p...,0.114565,sans hdr par choix de ne pas rédiger ce docume...,0.114558


In [15]:
merged_df.columns

Index(['SID',
       'Commentaires éventuels (sur le formulaire ou sur votre situation personnelle)',
       'Note personnelle',
       'Commentaires éventuels (sur le formulaire ou sur votre appréciation personnelle de l'enseignement)',
       'Note enseignement',
       'Commentaires éventuels (sur le formulaire ou sur votre appréciation personnelle de la recherche)',
       'Note recherche', 'Pouvez-vous expliquer vos réponses merci ...',
       'Note merci', 'Totalite des commentaires', 'Note totalite'],
      dtype='object')

In [16]:
merged_df.to_csv("resultat_analyse_sentiments_float.csv", index=False)

In [17]:
personnelle.to_csv("personnelle_float.csv", index=False)
enseignement.to_csv("enseignement_float.csv", index=False)
recherche.to_csv("recherche_float.csv", index=False)
merci.to_csv("merci_float.csv", index=False)
nouvelle_df.to_csv("totalite_float.csv", index=False)